<a href="https://colab.research.google.com/github/Frussen/ita-load-forecaster/blob/main/ItaLoadForecaster.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**ItaLoadForecaster**
##**End-to-End Italian Electricity Load Forecasting PoC**

Progetto per forecasting del carico elettrico orario in Italia usando dati open (Open Power System Data).  
Obiettivo: Dimostrare pipeline ML completa (data prep → modeling → MLOps → dashboard) applicabile a clienti enterprise energy/utility.

### Business Value
- Riduzione errori forecast → ottimizzazione balancing, peak shaving, riduzione costi energia  
- ROI simulato: stima savings € basati su prezzi storici  
- Scalabile a predictive maintenance / anomaly detection su grid data

### Tech Stack
- Data: Pandas, NumPy  
- Viz: Matplotlib, Seaborn  
- Modeling: Prophet (baseline), LSTM (advanced)  
- Tracking: MLflow  
- Deployment: Streamlit + HuggingFace Spaces  
- Environment: Google Colab (free GPU)

### Dataset
Open Power System Data - Time Series (IT load, prices, renewables generation)

### 1. Business Context & Objectives

Il business context è...

### 2. Data Ingestion

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt




### 3. Exploratory Data Analysis (EDA)


### 4. Feature Engineering


### 5. Baseline Model: Prophet


### 6. Advanced Model: LSTM


### 7. Model Evaluation & ROI Simulation


### 8. MLOps Tracking with MLflow


### 9. Deployment-ready Dashboard (Streamlit preview)


### 10. Next Steps & Production Path


### Appendix: Lessons Learned